<a href="https://colab.research.google.com/github/jderazoa/ML-con-PYTHON/blob/master/ncaaw_2021_h2o_automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ncaaw-2021-features-only/tourney.csv
/kaggle/input/ncaaw-2021-features-only/__results__.html
/kaggle/input/ncaaw-2021-features-only/__resultx__.html
/kaggle/input/ncaaw-2021-features-only/__notebook__.ipynb
/kaggle/input/ncaaw-2021-features-only/__output__.json
/kaggle/input/ncaaw-2021-features-only/test.csv
/kaggle/input/ncaaw-2021-features-only/custom.css
/kaggle/input/ncaaw-march-mania-2021/Conferences.csv
/kaggle/input/ncaaw-march-mania-2021/WNCAATourneyDetailedResults.csv
/kaggle/input/ncaaw-march-mania-2021/WSampleSubmissionStage1.csv
/kaggle/input/ncaaw-march-mania-2021/WRegularSeasonCompactResults.csv
/kaggle/input/ncaaw-march-mania-2021/WGameCities.csv
/kaggle/input/ncaaw-march-mania-2021/WNCAATourneySlots.csv
/kaggle/input/ncaaw-march-mania-2021/Cities.csv
/kaggle/input/ncaaw-march-mania-2021/WTeams.csv
/kaggle/input/ncaaw-march-mania-2021/WNCAATourneyCompactResults.csv
/kaggle/input/ncaaw-march-mania-2021/WSeasons.csv
/kaggle/input/ncaaw-march-mania-2021/WNCAAT

In [ ]:
# Starting H2O

import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

3.32.0.4
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6qr881k0
  JVM stdout: /tmp/tmp6qr881k0/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp6qr881k0/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_unknownUser_5hdfss
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
train = h2o.import_file('../input/ncaaw-2021-features-only/tourney.csv')
test = h2o.import_file('../input/ncaaw-2021-features-only/test.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
test.head()

ID,Season,TeamID1,TeamID2,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1,win_pct_A_2,win_pct_N_2,win_pct_H_2,win_pct_All_2,Score_A_1,relScore_A_1,Score_N_1,relScore_N_1,Score_H_1,relScore_H_1,Score_All_1,relScore_All_1,Score_A_2,relScore_A_2,Score_N_2,relScore_N_2,Score_H_2,relScore_H_2,Score_All_2,relScore_All_2,FGM_1,FGA_1,FGM3_1,FGA3_1,FTM_1,FTA_1,OR_1,DR_1,Ast_1,TO_1,Stl_1,Blk_1,PF_1,goal_rate_1,3p_goal_rate_1,ft_goal_rate_1,FGM_2,FGA_2,FGM3_2,FGA3_2,FTM_2,FTA_2,OR_2,DR_2,Ast_2,TO_2,Stl_2,Blk_2,PF_2,goal_rate_2,3p_goal_rate_2,ft_goal_rate_2,seed_num_1,seed_num_2,seed_num_diff
2015_3106_3107,2015,3106,3107,0.352941,1,0.666667,0.5,0.705882,1,0.769231,0.75,59.6471,-5.52941,69,10.5,60,5.88889,60.4286,-0.714286,65.9412,11.5882,64.5,29.5,74.6923,15.1538,69.4062,14.1562,20.4643,56.8214,4.85714,15.6786,14.6429,23.3214,14.3929,28.5714,11.8214,19.4286,6.85714,4.75,22.25,0.362812,0.317927,0.63866,26.1875,54.7188,2.0625,7.0625,14.9688,21.3125,13.25,23.375,15.25,16.7188,10.7188,1.75,18.3125,0.480176,0.305935,0.715335,15,13,2
2015_3106_3110,2015,3106,3110,0.352941,1,0.666667,0.5,0.714286,1,0.75,0.75,59.6471,-5.52941,69,10.5,60,5.88889,60.4286,-0.714286,61.5714,1.28571,65.5,5.5,64.625,10.4375,63.3438,6.125,20.4643,56.8214,4.85714,15.6786,14.6429,23.3214,14.3929,28.5714,11.8214,19.4286,6.85714,4.75,22.25,0.362812,0.317927,0.63866,23.6875,53.5312,4.46875,13.7812,11.5,15.875,8.59375,24.5,15.4062,12.9062,6.84375,2.84375,15.7812,0.443939,0.320191,0.718225,15,14,1
2015_3106_3113,2015,3106,3113,0.352941,1,0.666667,0.5,0.916667,0.5,0.833333,0.84375,59.6471,-5.52941,69,10.5,60,5.88889,60.4286,-0.714286,70.5833,13.8333,61.5,8,66.8889,11.7778,67.9375,12.3125,20.4643,56.8214,4.85714,15.6786,14.6429,23.3214,14.3929,28.5714,11.8214,19.4286,6.85714,4.75,22.25,0.362812,0.317927,0.63866,25,57.125,4.75,13.2812,13.1875,18.5938,13.3438,23.2812,14.6562,13.9062,8.09375,2.1875,15.2812,0.4377,0.355642,0.694855,15,3,12
2015_3106_3114,2015,3106,3114,0.352941,1,0.666667,0.5,0.8,1,0.933333,0.875,59.6471,-5.52941,69,10.5,60,5.88889,60.4286,-0.714286,62.8,10.4,74.5,11.5,66.5333,18.4667,65.2812,14.25,20.4643,56.8214,4.85714,15.6786,14.6429,23.3214,14.3929,28.5714,11.8214,19.4286,6.85714,4.75,22.25,0.362812,0.317927,0.63866,24.2188,54.2812,4.125,11,12.7188,19,11.9688,22.875,13.9688,14.1562,8.5625,2.6875,18.125,0.449274,0.353258,0.674205,15,11,4
2015_3106_3116,2015,3106,3116,0.352941,1,0.666667,0.5,0.454545,0.75,0.6,0.566667,59.6471,-5.52941,69,10.5,60,5.88889,60.4286,-0.714286,55.3636,-0.818182,64.75,4.5,62.7333,9.2,60.3,4.9,20.4643,56.8214,4.85714,15.6786,14.6429,23.3214,14.3929,28.5714,11.8214,19.4286,6.85714,4.75,22.25,0.362812,0.317927,0.63866,20.7667,55.7667,4.96667,16.3667,13.8,21.0333,13.3667,26.9333,11.8,13.4,6.4,3.56667,17.2333,0.375554,0.307348,0.66032,15,10,5
2015_3106_3124,2015,3106,3124,0.352941,1,0.666667,0.5,0.727273,1,1,0.909091,59.6471,-5.52941,69,10.5,60,5.88889,60.4286,-0.714286,72.8182,10.2727,76.8,12.2,85.7059,29.8824,80.0606,20.6667,20.4643,56.8214,4.85714,15.6786,14.6429,23.3214,14.3929,28.5714,11.8214,19.4286,6.85714,4.75,22.25,0.362812,0.317927,0.63866,30.5455,63.5455,3.27273,9.60606,15.697,22.5758,14.6667,29.8485,20.8182,14.0303,8.18182,5.69697,16.7576,0.48156,0.354295,0.692617,15,2,13
2015_3106_3129,2015,3106,3129,0.352941,1,0.666667,0.5,0.461538,0.75,0.846154,0.666667,59.6471,-5.52941,69,10.5,60,5.88889,60.4286,-0.714286,65.4615,1.53846,64,3.75,78,15.5385,70.7,7.9,20.4643,56.8214,4.85714,15.6786,14.6429,23.3214,14.3929,28.5714,11.8214,19.4286,6.85714,4.75,22.25,0.362812,0.317927,0.63866,24.9333,60.7333,8.33333,22.5,12.5,17.5333,14.8,26.1,15.5,17.5,7.8,4.33333,16.8333,0.409469,0.367344,0.72145,15,15,0
2015_3106_3140,2015,3106,3140,0.352941,1,0.666667,0.5,0.583333,0.8,0.785714,0.709677,59.6471,-5.52941,69,10.5,60,5.88889,60.4286,-0.714286,68.75,-1,68.8,5.4,68.0714,7.5,68.4516,3.87097,20.4643,56.8214,4.85714,15.6786,14.6429,23.3214,14.3929,28.5714,11.8214,19.4286,6.85714,4.75,22.25,0.362812,0.317927,0.63866,24.2581,55.9032,6.58065,18.4516,13

In [ ]:
train.head()

Season,TeamID1,TeamID2,result,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1,win_pct_A_2,win_pct_N_2,win_pct_H_2,win_pct_All_2,Score_A_1,relScore_A_1,Score_N_1,relScore_N_1,Score_H_1,relScore_H_1,Score_All_1,relScore_All_1,Score_A_2,relScore_A_2,Score_N_2,relScore_N_2,Score_H_2,relScore_H_2,Score_All_2,relScore_All_2,FGM_1,FGA_1,FGM3_1,FGA3_1,FTM_1,FTA_1,OR_1,DR_1,Ast_1,TO_1,Stl_1,Blk_1,PF_1,goal_rate_1,3p_goal_rate_1,ft_goal_rate_1,FGM_2,FGA_2,FGM3_2,FGA3_2,FTM_2,FTA_2,OR_2,DR_2,Ast_2,TO_2,Stl_2,Blk_2,PF_2,goal_rate_2,3p_goal_rate_2,ft_goal_rate_2,seed_num_1,seed_num_2,seed_num_diff
2003,3130,3330,1,0.75,0.5,0.75,0.714286,0.545455,0.333333,0.823529,0.677419,70.8333,9.58333,63,5,72.75,14,70.5357,10.8214,67.4545,1.18182,63.6667,6,70.3529,13.6471,68.6774,8.48387,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,5,12,-7
2003,3160,3140,1,0.75,0.666667,0.785714,0.758621,0.5,0.25,0.916667,0.633333,64.5833,5.08333,64.6667,7.33333,69.7143,11.5,67.069,8.41379,65.7143,2.42857,63,-2.25,69.8333,14.0833,67,6.46667,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,11,-5
2003,3208,3150,1,0.5,0,1,0.678571,0.5,0.666667,1,0.724138,73.9286,11.2857,65.5,-4,83.1667,17.6667,77.2857,12.9286,67.7143,0.642857,64.3333,2,78.4167,15.75,71.7931,7.03448,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,5,12,-7
2003,3261,3402,1,0.818182,1,0.933333,0.9,0.533333,0,0.714286,0.580645,74.7273,13.0909,80.25,17,77,22.8667,76.6,18.5,60.5333,-7,68.5,-1.5,66.5,7.35714,63.7419,-0.16129,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,16,-15
2003,3278,3408,1,0.733333,0.5,1,0.821429,0.615385,0,0.785714,0.678571,73.9333,7.13333,81,3.5,85.9091,25.2727,79.1429,14,71,8.46154,65,-13,66,10.7143,68.2857,8.82143,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,11,-5
2003,3280,3264,1,0.636364,0.666667,0.923077,0.766667,0.615385,1,0.714286,0.689655,73.5455,9.72727,71.1667,6.83333,78.3846,14.7692,75.1667,11.3333,60.2308,2.61538,68,11,65.6429,6.71429,63.3793,5.17241,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,14,-11
2003,3307,3274,1,0.363636,1,0.9375,0.733333,0.416667,0.666667,0.714286,0.586207,60.2727,1.18182,57,8.33333,72.375,19.125,66.4,11.4667,73.1667,-2.83333,74.3333,12.6667,72.7143,8.78571,73.069,4.37931,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,11,-5
2003,3314,3122,1,0.8,0.666667,0.9375,0.84375,0.785714,1,1,0.896552,70.1,3.2,75,3.83333,80.625,25.4375,76.2812,14.4375,69.5714,3.71429,76,15.6667,78,22.1667,73.7241,12.5862,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,14,-11
2003,3345,3434,1,0.583333,1,1,0.83871,0.363636,0.833333,0.636364,0.571429,70.9167,4.16667,69.8,12,77.8571,19.5714,73.871,12.3871,60.9091,-1.81818,65,5.5,63.7273,8.90909,62.8929,3.96429,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,15,-13
2003,3353,3443,1,0.642857,-1,0.846154,0.740741,0.538462,-1,0.882353,0.733333,65.2143,8.21429,0,0,70,13.8462,67.5185,10.9259,69,0.230769,0,0,75.2941,12.4706,72.5667,7.16667,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,13,-9


In [ ]:
x = test.columns[4:]
y = 'result'
train[y] = train[y].asfactor()

In [ ]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=10000, seed=47, max_runtime_secs=31000)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GLM_1_AutoML_20210311_214637,0.883005,0.423605,0.887515,0.205026,0.373931,0.139824
StackedEnsemble_BestOfFamily_AutoML_20210311_214637,0.880448,0.441166,0.883004,0.201058,0.377529,0.142528
StackedEnsemble_AllModels_AutoML_20210311_214637,0.876816,0.441739,0.875109,0.210317,0.378389,0.143179
GBM_5_AutoML_20210311_214637,0.873078,0.445444,0.87554,0.21164,0.381522,0.145559
GBM_grid__1_AutoML_20210311_214637_model_13,0.872452,0.445428,0.875746,0.23082,0.382206,0.146081
GBM_grid__1_AutoML_20210311_214637_model_1,0.87128,0.448812,0.874312,0.226852,0.383245,0.146877
GBM_grid__1_AutoML_20210311_214637_model_3,0.869334,0.450989,0.871453,0.218915,0.384914,0.148159
GBM_2_AutoML_20210311_214637,0.86805,0.458847,0.872363,0.232804,0.388863,0.151215
GBM_3_AutoML_20210311_214637,0.867811,0.45902,0.868677,0.223545,0.389096,0.151396
XGBoost_grid__1_AutoML_20210311_214637_model_6,0.867609,0.456848,0.868934,0.214286,0.385889,0.14891


In [ ]:
# The leader model is stored here
aml.leader

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_1_AutoML_20210311_214637


GLM Model: summary


,,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,Ridge ( lambda = 0.005906 ),"nlambda = 30, lambda.max = 31.29, lambda.min = 0.005906, lambda.1s...",59,59,38,automl_training_py_3_sid_882c




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.1324069189151769
RMSE: 0.363877615298299
LogLoss: 0.40397392676267063
Null degrees of freedom: 1511
Residual degrees of freedom: 1452
Null deviance: 2096.077074013276
Residual deviance: 1221.617154530316
AIC: 1341.617154530316
AUC: 0.8948307718148988
AUCPR: 0.8992090054871802
Gini: 0.7896615436297976

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5009977792571003: 


,,0,1,Error,Rate
0,0,618.0,138.0,0.1825,(138.0/756.0)
1,1,139.0,617.0,0.1839,(139.0/756.0)
2,Total,757.0,755.0,0.1832,(277.0/1512.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,deviance_xval,deviance_se,alpha,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2021-03-11 21:46:53,0.000 sec,2,.31E2,60,1.354798,NaN,1.360909,0.000869,0.0,0.363878,0.403974,0.470372,0.894831,0.899209,2.0,0.183201
1,,2021-03-11 21:46:53,0.018 sec,4,.19E2,60,1.337575,NaN,1.346732,0.001359,0.0,,,,,,,
2,,2021-03-11 21:46:53,0.032 sec,6,.12E2,60,1.312539,NaN,1.325760,0.002091,0.0,,,,,,,
3,,2021-03-11 21:46:53,0.055 sec,8,.75E1,60,1.277966,NaN,1.296009,0.003140,0.0,,,,,,,
4,,2021-03-11 21:46:53,0.069 sec,10,.47E1,60,1.233373,NaN,1.256355,0.004564,0.0,,,,,,,
5,,2021-03-11 21:46:53,0.078 sec,12,.29E1,60,1.180504,NaN,1.207399,0.006337,0.0,,,,,,,
6,,2021-03-11 21:46:53,0.090 sec,14,.18E1,60,1.123137,NaN,1.152156,0.008336,0.0,,,,,,,
7,,2021-03-11 21:46:53,0.099 sec,16,.11E1,60,1.065534,NaN,1.094885,0.010301,0.0,,,,,,,
8,,2021-03-11 21:46:53,0.107 sec,18,.69E0,60,1.011435,NaN,1.039725,0.012066,0.0,,,,,,,
9,,2021-03-11 21:46:53,0.115 sec,20,.43E0,60,0.963252,NaN,0.989756,0.013481,0.0,,,,,,,



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,seed_num_diff,0.704745,1.000000,0.068441
1,seed_num_1,0.581551,0.825193,0.056477
2,seed_num_2,0.581438,0.825034,0.056466
3,Ast_2,0.388466,0.551216,0.037726
4,Ast_1,0.388430,0.551164,0.037722
5,FGM_1,0.325179,0.461414,0.031580
6,FGM_2,0.324895,0.461010,0.031552
7,relScore_A_2,0.311835,0.442479,0.030284
8,relScore_A_1,0.311723,0.442321,0.030273
9,FGM3_2,0.240403,0.341121,0.023347



See the whole table with table.as_data_frame()


In [ ]:
h2o.remove(train)

In [ ]:
preds = aml.predict(test)
preds['p1'].as_data_frame().values.flatten().shape

glm prediction progress: |████████████████████████████████████████████████| 100%


(20160,)

In [ ]:
WSampleSubmission = pd.read_csv('../input/ncaaw-march-mania-2021/WSampleSubmissionStage1.csv')
WSampleSubmission.head()

,ID,Pred
0,2015_3106_3107,0.5
1,2015_3106_3110,0.5
2,2015_3106_3113,0.5
3,2015_3106_3114,0.5
4,2015_3106_3116,0.5


In [ ]:
test_preds = preds['p1'].as_data_frame().values
idx = test_preds.shape[0] //2
test_preds[idx:] = 1 - test_preds[idx:]

pred = pd.concat([test['ID'].as_data_frame(), pd.Series(test_preds.flatten())], axis=1).groupby('ID')[0]\
        .mean().reset_index().rename(columns={0:'Pred'})
sub = WSampleSubmission.drop(['Pred'],axis=1).merge(pred, on='ID')
sub.to_csv('h2o_automl_submission_1.csv', index=False)

In [ ]:
sub.head()

,ID,Pred
0,2015_3106_3107,0.152874
1,2015_3106_3110,0.364027
2,2015_3106_3113,0.031260
3,2015_3106_3114,0.140335
4,2015_3106_3116,0.194637
